In [1]:
using Distributed
addprocs(8)

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [2]:
# %%
@everywhere begin
    
using Pkg; Pkg.activate("..")
# Pkg.instantiate()
using GraphFusedLasso
# using Base.Threads
# using Revise
using Distributions
import StatsBase.weights
using Random
using RCall
using ProgressBars
using FileIO
using DataFrames, CSV
using Printf
# using Base.Threads
    
end

┌ Info: Precompiling GraphFusedLasso [ebf00d30-dbed-11e8-07b9-cfffd4a15404]
└ @ Base loading.jl:1186


      From worker 3:	
      From worker 6:	
      From worker 7:	
      From worker 6:	R version 3.4.3 (2017-11-30) -- "Kite-Eating Tree"
      From worker 6:	Copyright (C) 2017 The R Foundation for Statistical Computing
      From worker 6:	Platform: x86_64-w64-mingw32/x64 (64-bit)
      From worker 6:	
      From worker 3:	R version 3.4.3 (2017-11-30) -- "Kite-Eating Tree"
      From worker 3:	Copyright (C) 2017 The R Foundation for Statistical Computing
      From worker 3:	Platform: x86_64-w64-mingw32/x64 (64-bit)
      From worker 3:	
      From worker 7:	R version 3.4.3 (2017-11-30) -- "Kite-Eating Tree"
      From worker 7:	Copyright (C) 2017 The R Foundation for Statistical Computing
      From worker 7:	Platform: x86_64-w64-mingw32/x64 (64-bit)
      From worker 7:	
      From worker 9:	
      From worker 2:	
      From worker 5:	
      From worker 8:	
      From worker 4:	
      From worker 9:	R version 3.4.3 (2017-11-30) -- "Kite-Eating Tree"
      From worker 9:	Copyright (

In [3]:
println("Running with $(Threads.nthreads()) thread(s)")
# println("Running with $(nprocs()) process(ses)")

R"library('tidyverse')"

# Random.seed!(418916);

@everywhere begin

# %%
function generate_trace(task, N)
    cuts = [N ÷ 3, 2(N ÷ 3)]
#     cuts = sort(sample(2:N-1, 2, replace=false))
    x1 = 1:cuts[1]
    x2 = (cuts[1] + 1):cuts[2]
    x3 = (cuts[2] + 1):N
    x = [x1; x2; x3]
    
    values1 = 2.0(rand(Uniform(), 4) .- 0.5)

    if task == "smooth"
        y1 = values1[1] .+ (values1[2] - values1[1]) .* (x1 ./ cuts[1])
        y2 = values1[2] .+ (values1[3] - values1[2]) .* (x2 .- cuts[1]) ./ (cuts[2] - cuts[1])
        y3 = values1[3] .+ (values1[4] - values1[3]) .* (x3 .- cuts[2]) ./ (N - cuts[2])
    elseif task == "constant"
        y1 = fill(values1[1], cuts[1])
        y2 = fill(values1[2], cuts[2] - cuts[1])
        y3 = fill(values1[3], N - cuts[2])
    elseif task == "mixed"
        y1 = values1[1] .+ Int(rand() < 0.5) .* (values1[2] - values1[1]) .* (x1 ./ cuts[1])
        y2 = values1[2] .+ Int(rand() < 0.5) .* (values1[3] - values1[2]) .* (x2 .- cuts[1]) ./ (cuts[2] - cuts[1])
        y3 = values1[3] .+ Int(rand() < 0.5) .* (values1[4] - values1[3]) .* (x3 .- cuts[2]) ./ (N - cuts[2])
    else
        throw(ArgumentError)
    end
    μ = [y1; y2; y3]
    return μ
end


function generate_spt_task(task_space, task_time, N, pmiss; σ=0.3, outliers=false)
    μ1s = generate_trace(task_space, N)
    μ2s = generate_trace(task_space, N)
    ts =  generate_trace(task_time, N)
    
    evalpts = collect(range(-2.5, 2.5, length=100))
    
    μs = [(t + μ1, t + μ2) for (μ1, μ2) in zip(μ1s, μ2s), t in ts]
    dmodels = [MixtureModel([Normal(μ1, σ), Normal(μ2, σ)]) for (μ1, μ2) in μs]

    devals = [pdf.(d, evalpts) for d in dmodels];
    ndata = [sample([0, 10], weights([pmiss, 1.0 - pmiss])) for d in dmodels]
    y = [rand(d, n) for (d, n) in zip(dmodels, ndata)]
    
    if outliers
        Nobs = sum([1 for n in ndata if n > 0])
        K = Int(floor(Nobs * 0.5))
        idx = sample([i for (i, n) in enumerate(ndata) if n > 0], K, replace=false)
        for i in idx
            j = rand(1:length(y[i]))
            y[i][j] += rand([-1, 1]) * 5.0
        end
    end
                    
    # make matrix pts
    xrange = collect(1:N)
    # temporal
    ptr = Int[]
    brks = Int[1]
    for i in 1:N
        append!(ptr, xrange .+ (i - 1) * N)
        push!(brks, brks[end] + N)
    end
    istemporal = fill(true, N^2)
    # spatial
    xrange = [(i - 1) * N + 1 for i in 1:N]
    for i in 1:N
        append!(ptr, xrange .+ (i - 1))
        push!(brks, brks[end] + N)
    end
    append!(istemporal, fill(false, N^2))
    
    return Dict("evalpts" => evalpts,
                "dmodels" => dmodels,
                "devals" => devals,
                "y" => y,
                "ndata" => ndata,
                "mean1" => μ1s,
                "mean2" => μ2s,
                "t" => ts,
                "means" => μs,
                "ptr" => ptr,
                "brks" => brks,
                "istemporal" => istemporal)
end

# function for cross-validation fit

function generate_cvsets(y, nsplits)
    # make the cv splits
    N = length(y)
    cvsets = [Set{Int}() for i in 1:nsplits]
    iobs = shuffle([i for (i, yi) in enumerate(y) if !isempty(yi)])
    Nobs = length(iobs)
    splitsize = Nobs ÷ nsplits
    for k in 1:nsplits
        for i in ((k - 1) * splitsize + 1):(k * splitsize)
            push!(cvsets[k], iobs[i])
        end
    end
    return cvsets
end
               


function fit2(ytrain, ptr, brks, λ1, λ2, η1, η2, istemporal)
    N = length(ytrain)
    lambdasl1 = Float64[temp ? η1 : λ1 for temp in istemporal]
    lambdasl2 = Float64[temp ? η2 : λ2 for temp in istemporal]

    # create the tree
    M  = 33
    splits = collect(range(-2.5, 2.5, length=M))
    tree = DensityTree(splits)
    bins2counts = Dict()
    for (j, (li, ui)) in enumerate([tree.bins; [(i, i+1) for i in 1:M-1]])
        lower = splits[li]
        upper = splits[ui]
        k = [sum(lower .< yi .< upper) for yi in ytrain]
        bins2counts[(li, ui)] = k
    end
            
    # fit binomial model in each tree
    beta = zeros(N, M - 2)
    for j in 1:M - 2
        li, ui = tree.bins[j]
        mi = (ui + li) ÷ 2 
        parent_counts = bins2counts[(li, ui)] .+ 0.1
        left_counts = bins2counts[(li, mi)]  .+ 0.05

        level = trunc(Int, log2(j))

        model = BinomialEnet(
            ptr, brks,
            lambdasl1 .* 2.0 ^ (0.25 * level),
            lambdasl2 .* 2.0 ^ (0.25 * level);
            abstol=0.0,
            reltol=1e-3)
        fit!(model, left_counts, parent_counts; steps=100, parallel=false)

        beta[:, j] = model.beta
    end
    tree.beta = beta
    return tree
end
         
                
function cv_fit2(y, evalpts, ptr, brks, istemporal, lambdas, cvsets, models)
    # for each cv split get the mse error
    N = length(y)
    nsplits = length(cvsets)
    nlambdas = length(lambdas)
                    
    # prepare the tree structure and the bint 
    test_loglikelihood = pmap(1:nlambdas) do k
        λ1, λ2, η1, η2 = lambdas[k]
        Ntest = 0.0
        loglikelihood = 0.0
        for i = 1:nsplits
            # get the cv vector with missing data
            ytrain = [j in cvsets[i] ? Float64[] : yi
                      for (j, yi) in enumerate(y)]

            tree = fit2(ytrain, ptr, brks, λ1, λ2, η1, η2, istemporal)               
                            
            # compute the out-of-sample likelihood
            for j in collect(cvsets[i])
                test_eval = y[j]
                Ntest += length(test_eval)
                ll = - log.(predict(tree, sort(test_eval), j) .+ 1e-4)
                loglikelihood += sum(ll)
            end
        end
        loglikelihood / Ntest
    end
    # println(".")

    # now choose the best lambdas
#     display(collect(zip(test_loglikelihood, lambdas)))
    best_lambdas = lambdas[argmin(test_loglikelihood)]
    best_loglikelihood = minimum(test_loglikelihood)
    
    # compute validation likelihood
    nsims = 100
    samples = [rand(model, nsims) for model in models]
    λ1, λ2, η1, η2 = best_lambdas
    tree = fit2(y, ptr, brks, λ1, λ2, η1, η2, istemporal)
    lls = [- mean(log.(predict(tree, sort(x), j) .+ 1e-4)) for (j, x) in enumerate(samples)]
    validation_loglikelihood = mean(lls)

    return Dict("best_lambdas" => best_lambdas,
                "cv_loglikelihood" => best_loglikelihood,
                "val_loglikelihood" => validation_loglikelihood)
end
            
function get_hypers()
#     fl_choices = [1e-12, 0.33, 0.66, 1.0]
#     kal_choices = [1e-12, 0.75, 1.5, 3.0]
    fl_choices = [1e-6, 0.33, 0.66, 1.0] ./ 2.0
    kal_choices = [1e-6, 0.75, 1.5, 3.0] ./ 2.0
    lambdas_dict = Dict(
        "fl" =>  [(λ1, 1e-12, η1, 1e-12) for λ1 in fl_choices for η1 in fl_choices],
        "kal" => [(1e-12, λ2, 1e-12, η2) for λ2 in kal_choices for η2 in kal_choices],
        "enet" => [(λ1, λ2, η1, η2) for λ1 in fl_choices for λ2 in kal_choices
                                    for η1 in fl_choices for η2 in kal_choices])
    return lambdas_dict
end


function run_benchmarks(N, pmiss;
                        nsims=100,
                        nsplits=5,
                        tasks=("constant", "smooth", "mixed"),
                        outliers=false)
    experiment_results = []
    for task_space in tasks
        for task_time in tasks
            data = [
                generate_spt_task(task_space, task_time, N, pmiss, outliers=outliers)
                for _ in 1:nsims
            ]
            lambdas_dict = get_hypers()
            for method in ("fl", "kal", "enet")
                println("Running task_space $task_space task_time $task_time for method $method, outliers $outliers")
                
                new_records = []
                for (l, D) in ProgressBar(collect(enumerate(data)))
#                 for (l, D) in enumerate(data)
                    y = vec(D["y"])
                    models = vec(D["dmodels"])
                    ndata = vec(D["ndata"])
                    devals = vec(D["devals"])
                    ptr = D["ptr"]
                    brks = D["brks"]
                    evalpts = D["evalpts"]
                    istemporal = D["istemporal"]
                    
                    cvsets = generate_cvsets(y, nsplits)
                    lambdas = lambdas_dict[method]
                    results = cv_fit2(y, evalpts, ptr, brks, istemporal, lambdas, cvsets, models)

                    λ1, λ2, η1, η2 = results["best_lambdas"]
                    record = Dict(
                        :experiment => l,
                        :task_space => task_space,
                        :task_time => task_time,
                        :method => method,
                        :cv_ll => results["cv_loglikelihood"],
                        :val_ll => results["val_loglikelihood"],
                        :lamb1 => λ1,
                        :lamb2 => λ2,
                        :eta1 => η1,
                        :eta2 => η2,
                        :outliers => Int(outliers))
                    push!(new_records, record)
                    display(record)
                end
                append!(experiment_results, new_records)
            end
        end
    end
    return experiment_results
end

end

Running with 1 thread(s)


┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.4.4
│ -- Attaching packages --------------------------------------- tidyverse 1.2.1 --
│ v ggplot2 3.2.1     v purrr   0.2.4
│ v tibble  2.1.3     v dplyr   0.8.3
│ v tidyr   0.7.2     v stringr 1.2.0
│ v readr   1.1.1     v forcats 0.2.0
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x dplyr::filter() masks stats::filter()
│ x dplyr::lag()    masks stats::lag()
└ @ RCall C:\Users\mbg877\.julia\packages\RCall\iojZI\src\io.jl:113


In [4]:
# %%
# @everywhere begin
N = 30
nsims = 10
tasks = ("smooth", "constant", "mixed")
# end

pmiss = 0.8


experiment_results = run_benchmarks(N, pmiss, nsims=nsims, tasks=tasks)
experiment_results_out = run_benchmarks(N, pmiss, nsims=nsims, tasks=("mixed", ), outliers=true)

Running task_space smooth task_time smooth for method fl, outliers false


Dict{Symbol,Any} with 11 entries:
  :eta2       => 1.0e-12
  :lamb1      => 0.132
  :lamb2      => 1.0e-12
  :method     => "fl"
  :experiment => 1
  :eta1       => 0.2
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.827946
  :val_ll     => 0.772092
  :task_time  => "smooth"

0.00%┣                                                             ┫ 0/10 Inf:Inf<Inf:Inf, 0.00 it/s]

Dict{Symbol,Any} with 11 entries:
  :eta2       => 1.0e-12
  :lamb1      => 0.2
  :lamb2      => 1.0e-12
  :method     => "fl"
  :experiment => 2
  :eta1       => 0.2
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.774578
  :val_ll     => 0.676679
  :task_time  => "smooth"

10.00%┣██████▏                                                       ┫ 1/10 00:40<Inf:Inf, 0.00 it/s]

...

Dict{Symbol,Any} with 11 entries:
  :eta2       => 0.3
  :lamb1      => 2.0e-7
  :lamb2      => 0.6
  :method     => "enet"
  :experiment => 9
  :eta1       => 2.0e-7
  :task_space => "mixed"
  :outliers   => 1
  :cv_ll      => 1.1669
  :val_ll     => 0.702614
  :task_time  => "mixed"

80.00%┣███████████████████████████████████████████████████▏            ┫ 8/10 28:04<08:01, 0.00 it/s]

Dict{Symbol,Any} with 11 entries:
  :eta2       => 0.6
  :lamb1      => 2.0e-7
  :lamb2      => 0.3
  :method     => "enet"
  :experiment => 10
  :eta1       => 2.0e-7
  :task_space => "mixed"
  :outliers   => 1
  :cv_ll      => 1.23036
  :val_ll     => 0.800187
  :task_time  => "mixed"

100.00%┣██████████████████████████████████████████████████████████████┫ 10/10 31:18<00:00, 0.00 it/s]


30-element Array{Any,1}:
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.2,:lamb2=>1.0e-12,:method=>"fl",:experiment=>1,:eta1=>0.2,:task_space=>"mixed",:outliers=>1,:cv_ll=>1.09326,:val_ll=>0.625121…)   
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.2,:lamb2=>1.0e-12,:method=>"fl",:experiment=>2,:eta1=>0.2,:task_space=>"mixed",:outliers=>1,:cv_ll=>1.44618,:val_ll=>0.991524…)   
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.2,:lamb2=>1.0e-12,:method=>"fl",:experiment=>3,:eta1=>0.2,:task_space=>"mixed",:outliers=>1,:cv_ll=>1.30604,:val_ll=>0.878497…)   
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.066,:lamb2=>1.0e-12,:method=>"fl",:experiment=>4,:eta1=>0.2,:task_space=>"mixed",:outliers=>1,:cv_ll=>1.35527,:val_ll=>0.897577…) 
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.2,:lamb2=>1.0e-12,:method=>"fl",:experiment=>5,:eta1=>0.2,:task_space=>"mixed",:outliers=>1,:cv_ll=>1.03665,:val_ll=>0.574766…)   
 Dict{Symbol,Any}(:eta2=>1.0e-12,:lamb1=>0.132,:lamb2=>1.0e-12,:method=>"fl",:experiment=>6,:eta1=>0.2,:t

In [5]:

# %%
df = DataFrame(experiment = Int[],
               task_space=String[],
               task_time=String[],
               method=String[],
               cv_ll=Float64[],
               val_ll=Float64[],
               lamb1=Float64[],
               lamb2=Float64[],
               eta1=Float64[],
               eta2=Float64[],
               outliers=Int[])
for record in [experiment_results; experiment_results_out]
    push!(df, record)
end

CSV.write("benchmarks-results-spt_p08_scaled.csv", df)

# %%
head(df, 10)


# %%
res = R"""
df = $df %>% 
    group_by(task_space, task_time, method, outliers) %>%
    summarize(val_loglikelihood = mean(val_ll),
              cv_loglikelihood = mean(cv_ll))
df
"""
res = rcopy(res)

# %%



,task_space,task_time,method,outliers,val_loglikelihood,cv_loglikelihood
,String,String,String,Int64,Float64,Float64
1,constant,constant,enet,0,0.716805,0.747859
2,constant,constant,fl,0,0.742836,0.799245
3,constant,constant,kal,0,0.735285,0.772055
4,constant,mixed,enet,0,0.733996,0.758068
5,constant,mixed,fl,0,0.763687,0.804908
6,constant,mixed,kal,0,0.736812,0.761795
7,constant,smooth,enet,0,0.70014,0.726449
8,constant,smooth,fl,0,0.733509,0.781413
9,constant,smooth,kal,0,0.70646,0.737081


In [6]:
print(df)

300×11 DataFrame
│ Row │ experiment │ task_space │ task_time │ method │ cv_ll    │ val_ll   │ lamb1   │ lamb2   │ eta1    │ eta2    │ outliers │
│     │ Int64      │ String     │ String    │ String │ Float64  │ Float64  │ Float64 │ Float64 │ Float64 │ Float64 │ Int64    │
├─────┼────────────┼────────────┼───────────┼────────┼──────────┼──────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ 1   │ 1          │ smooth     │ smooth    │ fl     │ 0.827946 │ 0.772092 │ 0.132   │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 2   │ 2          │ smooth     │ smooth    │ fl     │ 0.774578 │ 0.676679 │ 0.2     │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 3   │ 3          │ smooth     │ smooth    │ fl     │ 1.01038  │ 1.01173  │ 0.066   │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 4   │ 4          │ smooth     │ smooth    │ fl     │ 0.779746 │ 0.744549 │ 0.2     │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 5   │ 5          │ smooth     │ smooth    │ fl     │ 1.04391  │ 1.00248  │ 0.132   │ 

In [ ]:
# %%
# @everywhere begin
N = 30
nsims = 10
tasks = ("smooth", "constant", "mixed")
# end

pmiss = 0.1


experiment_results = run_benchmarks(N, pmiss, nsims=nsims, tasks=tasks)
experiment_results_out = run_benchmarks(N, pmiss, nsims=nsims, tasks=("mixed", ), outliers=true)

Running task_space smooth task_time smooth for method fl, outliers false


Dict{Symbol,Any} with 11 entries:
  :eta2       => 1.0e-12
  :lamb1      => 0.132
  :lamb2      => 1.0e-12
  :method     => "fl"
  :experiment => 1
  :eta1       => 0.132
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.438047
  :val_ll     => 0.478896
  :task_time  => "smooth"

0.00%┣                                                             ┫ 0/10 Inf:Inf<Inf:Inf, 0.00 it/s]

Dict{Symbol,Any} with 11 entries:
  :eta2       => 1.0e-12
  :lamb1      => 0.2
  :lamb2      => 1.0e-12
  :method     => "fl"
  :experiment => 2
  :eta1       => 0.132
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.57376
  :val_ll     => 0.604859
  :task_time  => "smooth"

10.00%┣██████▏                                                       ┫ 1/10 00:28<Inf:Inf, 0.00 it/s]

...

Dict{Symbol,Any} with 11 entries:
  :eta2       => 0.3
  :lamb1      => 0.066
  :lamb2      => 0.15
  :method     => "enet"
  :experiment => 1
  :eta1       => 2.0e-7
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.421451
  :val_ll     => 0.439675
  :task_time  => "smooth"

0.00%┣                                                             ┫ 0/10 Inf:Inf<Inf:Inf, 0.00 it/s]

Dict{Symbol,Any} with 11 entries:
  :eta2       => 0.15
  :lamb1      => 2.0e-7
  :lamb2      => 0.3
  :method     => "enet"
  :experiment => 2
  :eta1       => 0.066
  :task_space => "smooth"
  :outliers   => 0
  :cv_ll      => 0.553766
  :val_ll     => 0.584227
  :task_time  => "smooth"

10.00%┣██████▏                                                       ┫ 1/10 07:03<Inf:Inf, 0.00 it/s]

InterruptException: InterruptException:

In [8]:

# %%
df = DataFrame(experiment = Int[],
               task_space=String[],
               task_time=String[],
               method=String[],
               cv_ll=Float64[],
               val_ll=Float64[],
               lamb1=Float64[],
               lamb2=Float64[],
               eta1=Float64[],
               eta2=Float64[],
               outliers=Int[])
for record in [experiment_results; experiment_results_out]
    push!(df, record)
end

CSV.write("benchmarks-results-spt_p01_scaled.csv", df)

# %%
head(df, 10)


# %%
res = R"""
df = $df %>% 
    group_by(task_space, task_time, method, outliers) %>%
    summarize(val_loglikelihood = mean(val_ll),
              cv_loglikelihood = mean(cv_ll))
df
"""
res = rcopy(res)

# %%



,task_space,task_time,method,outliers,val_loglikelihood,cv_loglikelihood
,String,String,String,Int64,Float64,Float64
1,constant,constant,enet,0,0.716805,0.747859
2,constant,constant,fl,0,0.742836,0.799245
3,constant,constant,kal,0,0.735285,0.772055
4,constant,mixed,enet,0,0.733996,0.758068
5,constant,mixed,fl,0,0.763687,0.804908
6,constant,mixed,kal,0,0.736812,0.761795
7,constant,smooth,enet,0,0.70014,0.726449
8,constant,smooth,fl,0,0.733509,0.781413
9,constant,smooth,kal,0,0.70646,0.737081


In [9]:
print(df)

300×11 DataFrame
│ Row │ experiment │ task_space │ task_time │ method │ cv_ll    │ val_ll   │ lamb1   │ lamb2   │ eta1    │ eta2    │ outliers │
│     │ Int64      │ String     │ String    │ String │ Float64  │ Float64  │ Float64 │ Float64 │ Float64 │ Float64 │ Int64    │
├─────┼────────────┼────────────┼───────────┼────────┼──────────┼──────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ 1   │ 1          │ smooth     │ smooth    │ fl     │ 0.827946 │ 0.772092 │ 0.132   │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 2   │ 2          │ smooth     │ smooth    │ fl     │ 0.774578 │ 0.676679 │ 0.2     │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 3   │ 3          │ smooth     │ smooth    │ fl     │ 1.01038  │ 1.01173  │ 0.066   │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 4   │ 4          │ smooth     │ smooth    │ fl     │ 0.779746 │ 0.744549 │ 0.2     │ 1.0e-12 │ 0.2     │ 1.0e-12 │ 0        │
│ 5   │ 5          │ smooth     │ smooth    │ fl     │ 1.04391  │ 1.00248  │ 0.132   │ 